# **Starter Code**
- Imports
- Checking GPU availability
- Getting the Kaggle set and description of the download




In [ ]:

# !pip install -q ultralytics kagglehub pyyaml opencv-python efficientnet_pytorch torch torchvision
import torch
import numpy as np
import kagglehub
import pathlib
import yaml
import shutil
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from pathlib import Path
from sklearn.metrics import confusion_matrix
import seaborn as sns
import cv2
import random



In [ ]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():     # Check for T4 availability on Colab (DON'T CHANGE)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected. Go to Runtime -> Change runtime type ->  GPU")

In [ ]:

path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")      # Getting the DATASET from kagglehub via CLI API (see their website - DON'T CHANGE)

local_path = pathlib.Path("/content/datasets/crop-pests")     # Saved the images to a local path to increase efficiency
local_path.parent.mkdir(parents=True, exist_ok=True)
shutil.copytree(path, local_path, dirs_exist_ok=True)

data_yaml_path = local_path / "data.yaml"     # YAML CONGIF (DON'T CHANGE)
data_cfg = {
    "path": str(local_path),
    "train": "train/images",
    "val":   "valid/images",
    "test":  "test/images",
    "nc": 12,
    "names":
     [
      "Ants",
      "Bees",
      "Beetles",
      "Catterpillars",     # NOTE: HAD TO CHANGE SPELLING BECAUSE DATASET IMAGES ARE SPELT LIKE THIS
      "Earthworms",
      "Earwigs",
      "Grasshoppers",
      "Moths",
      "Slugs",
      "Snails",
      "Wasps",
      "Weevils",
    ]
}

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f)


# CHECKS
num_train_images = len(list((local_path / "train" / "images").glob("*.jpg")))
num_val_images = len(list((local_path / "valid" / "images").glob("*.jpg")))
num_test_images = len(list((local_path / "test" / "images").glob("*.jpg")))

num_train_labels = len(list((local_path / "train" / "labels").glob("*.txt")))
num_val_labels = len(list((local_path / "valid" / "labels").glob("*.txt")))
num_test_labels = len(list((local_path / "test" / "labels").glob("*.txt")))

print(f"Number of training images: {num_train_images}")
print(f"Number of validation images: {num_val_images}")
print(f"Number of test images: {num_test_images}")
print(f"Number of training labels: {num_train_labels}")
print(f"Number of validation labels: {num_val_labels}")
print(f"Number of test labels: {num_test_labels}")


# **Our Evaluation Metrics**
- mAP@0.5 >= 0.5
- mAP@0.5:0.95 (elps you detect over/under-sized bounding boxes)
- Precision (false positive control)
- Recall (false negative control)
- F1 score
- Training and Testing times



In [ ]:
def base_evaluation_metrics(val_results):
    metrics = val_results.results_dict
    precision = metrics.get('metrics/precision(B)', 0)
    recall    = metrics.get('metrics/recall(B)', 0)
    mAP50     = metrics.get('metrics/mAP50(B)', 0)
    mAP50_95  = metrics.get('metrics/mAP50-95(B)', 0)
    f1_score  = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0
    print(f"\nMean Average Precision (mAP@0.5):        {mAP50:.4f}")
    print(f"Mean Average Precision (mAP@0.5:0.95):   {mAP50_95:.4f}")
    print(f"Precision: {precision:.4f}  |  Recall: {recall:.4f}  |  F1-score: {f1_score:.4f}")

In [ ]:
def format_time(seconds):
    mins, secs = divmod(seconds, 60)      # Converts total seconds into minutes and format.
    return f"{int(mins):0d}m {secs:.2f}s"


# Noise and Filter/Blur CHECK
--------------------------------------------------------
- Gaussian Noise
- Salt & Pepper
- Gaussian Blur

In [ ]:
# Noise and filter constants
APPLY_CORRUPTION = True
CORRUPT_TYPE = 'gaussian_blur'   # one of: 'gaussian_noise', 'salt_pepper_noise', 'gaussian_blur'
CORRUPT_STRENGTH = 0.2

In [ ]:

# Noise and filter/blur analysis
def apply_corruption_to_folder(source_img_dir, destination_img_dir, corruption_type, strength=0.01):
    source_lbl_dir = source_img_dir.parent / "labels"     # Determine corresponding label directories
    destination_lbl_dir = destination_img_dir.parent / "labels"

    if destination_img_dir.exists():      # Remove existing destination directories if they exist
        shutil.rmtree(destination_img_dir)
    if destination_lbl_dir.exists():
        shutil.rmtree(destination_lbl_dir)

    shutil.copytree(source_img_dir, destination_img_dir)       # Copy images and labels
    if source_lbl_dir.exists():
        shutil.copytree(source_lbl_dir, destination_lbl_dir)


    if corruption_type == 'gaussian_noise':
        sigma = int(strength * 255)
        print(f"\nApplying {corruption_type} (Sigma: {sigma}) to images in {destination_img_dir.name}...")

        for img_file in destination_img_dir.glob('*.jpg'):      # Adjust extension if needed
            img = cv2.imread(str(img_file))
            if img is None:
                continue

            img_f = img.astype(np.float32)
            noise = np.random.normal(0, sigma, img.shape).astype(np.float32)
            corrupted_img = img_f + noise
            corrupted_img = np.clip(corrupted_img, 0, 255).astype(np.uint8)
            cv2.imwrite(str(img_file), corrupted_img)

    elif corruption_type == 'salt_pepper_noise':
        ratio = strength      # Ratio of pixels to corrupt
        print(f"\nApplying {corruption_type} (Ratio: {ratio}) to images in {destination_img_dir.name}...")

        for img_file in destination_img_dir.glob('*.jpg'):      # Adjust extension if needed
            img = cv2.imread(str(img_file))
            if img is None:
                continue

            corrupted_img = img.copy()
            h, w, c = img.shape
            num_pixels = int(ratio * h * w)

            # Salt (white)
            ys = np.random.randint(0, h, num_pixels)
            xs = np.random.randint(0, w, num_pixels)
            corrupted_img[ys, xs, :] = 255

            # Pepper (black)
            ys = np.random.randint(0, h, num_pixels)
            xs = np.random.randint(0, w, num_pixels)
            corrupted_img[ys, xs, :] = 0

            cv2.imwrite(str(img_file), corrupted_img)

    elif corruption_type == 'gaussian_blur':
        ksize = int(strength * 20)      # scale to 1–20
        ksize = max(1, ksize)
        if ksize % 2 == 0:
            ksize += 1

        print(f"\nApplying {corruption_type} (ksize: {ksize}) to images in {destination_img_dir.name}...")

        for img_file in destination_img_dir.glob('*.jpg'):
            img = cv2.imread(str(img_file))
            if img is None:
                continue

            corrupted_img = cv2.GaussianBlur(img, (ksize, ksize), 0)
            cv2.imwrite(str(img_file), corrupted_img)


if (APPLY_CORRUPTION):
  original_val_path = local_path / "valid" / "images"
  corrupt_val_path = local_path / "valid_noisy" / "images"      # Create a new validation directory for the corrupted test set

  apply_corruption_to_folder(
      original_val_path,
      corrupt_val_path,
      CORRUPT_TYPE,
      CORRUPT_STRENGTH
  )


  data_cfg_corrupted = data_cfg.copy()
  data_cfg_corrupted['val'] = "valid_noisy/images"
  data_yaml_path_corrupted = local_path / "data_corrupted.yaml"      # Update the YAML to point to the corrupted validation set for the experiment

  # CHECK
  with open(data_yaml_path_corrupted, "w") as f:
      yaml.safe_dump(data_cfg_corrupted, f)
  print("Wrote corrupted config:", data_yaml_path_corrupted)

In [ ]:
model = YOLO("yolov8n.pt")

start = time.time()
train_res = model.train(
    data=str(data_yaml_path_corrupted if APPLY_CORRUPTION else data_yaml_path),
    epochs=10,
    imgsz=640,
    batch=-1,
    device=0,
    cache=True,
    amp=True,
    project="pests_fast",
    name="yolov8n_colab",
    plots=False
)

end = time.time()
train_time = end - start


start = time.time()
val_results = model.val(
    data=str(data_yaml_path_corrupted if APPLY_CORRUPTION else data_yaml_path),     # Ensure this points to the corrupted set if APPLY_CORRUPTION is True
    split="val",
    batch=16,
    iou=0.5,
    device=0,
    workers=2
)

end = time.time()
val_time = end - start


base_evaluation_metrics(val_results)
print(f"Training Time (Total): {format_time(train_time)}")
print(f"Testing/Validation Time (Total): {format_time(val_time)}")


# Resolution and Mosaic testing

------------------------------------------

- image_size_list = [64, 128, 256, 512, 640]
- mosaic_list = [ 0.2, 0.4, 0.6, 0.8]



In [ ]:
# Load the hypertuned YOLOv8n model

def call_model_1(epoch_size=10, image_size=640, mosaic_amount=1.0):
  model= YOLO("yolov8n.pt")
  start = time.time()
  train_res = model.train(
      data=str(data_yaml_path),
      batch=-1,
      device=0,
      amp=True,

      epochs=epoch_size,
      imgsz=image_size,
      mosaic=mosaic_amount,
      cache=True,


      project="pests_fast",
      name="yolov8n_wiou_like",
  )


  end = time.time()
  train_time = end - start

  start = time.time()
  val_results = model.val(
      data=str(data_yaml_path),
      split="val",
      batch=16,
      device=0,
      iou = 0.5,
      workers=2
  )

  end = time.time()
  val_time = end - start

  base_evaluation_metrics(val_results)
  print(f"Training Time (Total): {format_time(train_time)}")
  print(f"Testing/Validation Time (Total): {format_time(val_time)}")


In [ ]:
image_size_list = [64, 128, 256, 512, 640]

# DUE TO RAM ISSUES, CALLED SEPARATELY.
# Running on contsant 10 epochs for time

# call_model_1(image_size=image_size_list[0])
# call_model_1(image_size=image_size_list[1])
# call_model_1(image_size=image_size_list[2])
# call_model_1(image_size=image_size_list[3])
call_model_1(image_size=image_size_list[4])



**call_model_1(image_size=image_size_list[0])**
- Mean Average Precision (mAP@0.5):        0.5145
Mean Average Precision (mAP@0.5:0.95):   0.2505
Precision: 0.5544  |  Recall: 0.4849  |  F1-score: 0.5173
Training Time (Total): 9m 34.87s
Testing/Validation Time (Total): 0m 8.76s

**call_model_1(image_size=image_size_list[1])**
Mean Average Precision (mAP@0.5):        0.6577
Mean Average Precision (mAP@0.5:0.95):   0.3768
Precision: 0.7324  |  Recall: 0.5873  |  F1-score: 0.6519
Training Time (Total): 9m 53.36s
Testing/Validation Time (Total): 0m 9.02s

**call_model_1(image_size=image_size_list[2])**
Mean Average Precision (mAP@0.5):        0.7382
Mean Average Precision (mAP@0.5:0.95):   0.4342
Precision: 0.8095  |  Recall: 0.6682  |  F1-score: 0.7321
Training Time (Total): 9m 51.80s
Testing/Validation Time (Total): 0m 9.22s

**call_model_1(image_size=image_size_list[3])**
Mean Average Precision (mAP@0.5):        0.7359
Mean Average Precision (mAP@0.5:0.95):   0.4249
Precision: 0.8050  |  Recall: 0.6800  |  F1-score: 0.7372
Training Time (Total): 5m 9.23s
Testing/Validation Time (Total): 0m 9.47s

**call_model_1(image_size=image_size_list[4])**
Mean Average Precision (mAP@0.5):        0.7248
Mean Average Precision (mAP@0.5:0.95):   0.4099
Precision: 0.7818  |  Recall: 0.6710  |  F1-score: 0.7222
Training Time (Total): 6m 43.25s
Testing/Validation Time (Total): 0m 10.78s

In [ ]:
mosaic_list = [0.2, 0.4, 0.6, 0.8]

# DUE TO RAM ISSUES, CALLED SEPARATELY.
# Running on contsant 10  epochs for time

# call_model_1(mosaic_amount=mosaic_list[0])
# call_model_1(mosaic_amount=mosaic_list[1])
# call_model_1(mosaic_amount=mosaic_list[2])
call_model_1(mosaic_amount=mosaic_list[3])


**call_model_1(mosaic_amount=mosaic_list[0])**
Mean Average Precision (mAP@0.5):        0.7339
Mean Average Precision (mAP@0.5:0.95):   0.4147
Precision: 0.8058  |  Recall: 0.6660  |  F1-score: 0.7293
Training Time (Total): 6m 35.93s
Testing/Validation Time (Total): 0m 10.92s

**call_model_1(mosaic_amount=mosaic_list[1])**
Mean Average Precision (mAP@0.5):        0.7248
Mean Average Precision (mAP@0.5:0.95):   0.4099
Precision: 0.7818  |  Recall: 0.6710  |  F1-score: 0.7222
Training Time (Total): 6m 35.80s
Testing/Validation Time (Total): 0m 11.17s

**call_model_1(mosaic_amount=mosaic_list[2])**
Mean Average Precision (mAP@0.5):        0.7339
Mean Average Precision (mAP@0.5:0.95):   0.4147
Precision: 0.8058  |  Recall: 0.6660  |  F1-score: 0.7293
Training Time (Total): 6m 34.23s
Testing/Validation Time (Total): 0m 10.76s

**call_model_1(mosaic_amount=mosaic_list[3])**
Mean Average Precision (mAP@0.5):        0.7248
Mean Average Precision (mAP@0.5:0.95):   0.4099
Precision: 0.7818  |  Recall: 0.6710  |  F1-score: 0.7222
Training Time (Total): 6m 35.59s
Testing/Validation Time (Total): 0m 11.05s

